In [11]:
import pandas as pd
import numpy as np
import os

main_dir = os.environ.get("MAIN_DIRECTORY")
os.chdir(main_dir)

from preprocessing.preprocessor import preprocess_audio

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
model_save_path = os.environ.get("PRETRAINED_MODELS_PATH")

In [3]:
sentiments = ['negative', 'neutral', 'positive']
print(sentiments)

['negative', 'neutral', 'positive']


In [4]:
data = preprocess_audio("models/audio_model/data")

In [5]:
data.head()

,Name,Features,Label
0,03-01-08-02-02-02-23.wav,"[[-3.4641016, -3.4641016, -3.4641016, -3.46410...",0
1,03-01-08-02-02-02-23.wav,"[[-3.464102, -3.464102, -3.464102, -3.464102, ...",0
2,03-01-08-02-02-02-23.wav,"[[-3.4641018, -3.4641018, -3.4641018, -3.46410...",0
3,03-01-08-02-02-02-23.wav,"[[-3.4641013, -3.4641013, -3.4641013, -3.46410...",0
4,03-01-08-02-02-02-23.wav,"[[-3.4641001, -3.463962, -3.4638703, -3.464023...",0


In [ ]:
X = np.array(data["Features"].to_list())
y = to_categorical(data['Label'], num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [40]:
model = Sequential([
    Conv1D(2048, 5, activation='relu', input_shape=(165, 13)),
    MaxPooling1D(3),
    Conv1D(512, 5, activation='relu'),
    MaxPooling1D(3),
    Conv1D(128, 3, activation='relu'),
    GlobalAveragePooling1D(),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

In [41]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 161, 2048)         135168    
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 53, 2048)         0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 49, 512)           5243392   
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 16, 512)          0         
 g1D)                                                            
                                                                 
 conv1d_23 (Conv1D)          (None, 14, 128)           196736    
                                                                 
 global_average_pooling1d_7   (None, 128)             

In [42]:
opt = Adam(learning_rate=0.00005)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [43]:
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/20
60/60 [==============================] - 20s 329ms/step - loss: 0.9352 - accuracy: 0.4467 - val_loss: 0.8900 - val_accuracy: 0.6250
Epoch 2/20
60/60 [==============================] - 19s 317ms/step - loss: 0.8901 - accuracy: 0.5094 - val_loss: 0.8888 - val_accuracy: 0.4583
Epoch 3/20
60/60 [==============================] - 17s 291ms/step - loss: 0.8755 - accuracy: 0.5408 - val_loss: 0.8899 - val_accuracy: 0.4708
Epoch 4/20
60/60 [==============================] - 20s 335ms/step - loss: 0.8664 - accuracy: 0.5471 - val_loss: 0.8518 - val_accuracy: 0.5792
Epoch 5/20
60/60 [==============================] - 24s 392ms/step - loss: 0.8545 - accuracy: 0.5439 - val_loss: 0.8364 - val_accuracy: 0.6167
Epoch 6/20
60/60 [==============================] - 17s 287ms/step - loss: 0.8444 - accuracy: 0.5628 - val_loss: 0.8454 - val_accuracy: 0.5375
Epoch 7/20
60/60 [==============================] - 19s 310ms/step - loss: 0.8449 - accuracy: 0.5722 - val_loss: 0.8336 - val_accuracy: 0.5833

In [44]:
model_json = model.to_json()
with open(os.path.join(model_save_path, "audio_model.json"), "w") as file:
    file.write(model_json)

In [45]:
model.save('data/pretrained_models/audio_model.h5')